In [1]:
import re
from pandas import Series, DataFrame
import pandas as pd
import json
import numpy as np
import string

In [2]:
# Generate the qapair.txt 
# File qapair.txt is in this format : 
# Question_id , Answer0_vote , Answer1_vote , ... , NumOfAnswer
def generate_qapair(PATH_IN, PATH_OUT):
    
    # f_in  = open(PATH_IN + 'answer.json', 'r')
    # f_out = open(PATH_OUT + 'qapair.txt', 'w') 
    
    with open(PATH_IN + 'answer.json' , 'r') as f_in, \
         open(PATH_OUT + 'qapair.txt' , 'w') as f_out :
            
            line1 = f_in.readline()
            info1 = line1.split('"')
            pre_qid = info1[3]
            pre_vote = info1[-2]
            f_out.write(pre_qid+','+pre_vote)

            num_vote = 0
            for line in f_in:
                info = line.split('"')
                cur_qid = info[3]
                # print cur_qid
                cur_vote = info[-2]
                if cur_qid == pre_qid :
                    f_out.write(','+cur_vote)
                    num_vote = num_vote + 1
                else :
                    f_out.write(','+str(num_vote + 1))
                    num_vote = 0
                    f_out.write('\n'+cur_qid+','+cur_vote)
                pre_qid = cur_qid


In [3]:
# extract numOfAnswer >= 3 qid write to qid_3more
# extract numOfAnswer < 3 qid write to qid_3less
def extract_qid(PATH_IN, PATH_OUT) :

    #f_in = open(PATH_IN + 'qapair.txt', 'r')
    #f_out = open(PATH_OUT + 'qid_ex.txt', 'w')    
    
    with open(PATH_IN + 'qapair.txt', 'r') as f_in, \
    open(PATH_OUT + 'qid_3more.txt', 'w') as f_out1,     \
    open(PATH_OUT + 'qid_3less.txt', 'w') as f_out2:
        examples = 0
        for line in f_in:

            info = line.split(',')
            qid = info[0]
            num_answer = info[-1]

            if int(num_answer) >= 3:
                # print num_a
                # examples = examples + 1   
                best_answer_votes = int( info[1] )
                second_answer_votes = int( info[2] )
                
                # drop the answers which answer_votes == 0  
                if best_answer_votes > 0 and best_answer_votes > second_answer_votes : 
                    f_out1.write(qid+'\n')
                    # print qid`
            else :
                f_out2.write(qid+'\n')

In [4]:
# split question.json into 
# question_train.json (numOfAnswer >= 3 )
# question_test.sjon (numOfAnswer < 3 )
def split_qustion(PATH_IN, PATH_OUT) :

    with open(PATH_IN + 'qid_3more.txt', 'r') as f,         \
        open(PATH_IN + 'question.json', 'r') as f_in,      \
        open(PATH_IN + 'question_train.json', 'w') as f_out1, \
        open(PATH_IN + 'question_test.json', 'w') as f_out2: 
        qid_list = [line.strip() for line in f]

        # f_in = open(PATH_IN + 'question.json', 'r')
        # f_out = open(PATH_OUT + 'question_filtered.json', 'w')

        for line in f_in:
            info = line.split('"')
            qid = info[-2]
            if qid in qid_list:
                # print qid
                f_out1.write(line)
            else :
                f_out2.write(line)

In [5]:
def format_train_data(PATH_IN, PATH_OUT, FILE):
    
    with open(PATH_IN + 'answer.json', 'r') as f1, \
        open(PATH_IN + 'question_train.json', 'r') as f2, \
        open(PATH_IN + 'user.json', 'r') as f3, \
        open(PATH_IN + 'qtags.json', 'r') as f4 :
    
            answer = [json.loads(line.strip().strip(',')) for line in f1]
            #print answer[0]
            #answer_frame 
            answer_frame = DataFrame(answer)
            #answer_frame = answer_frame.drop_duplicates('answer_content')
            answer_frame = answer_frame.drop_duplicates('answer_id')
            #print answer_frame #14464 x 5

            question = [json.loads(line.strip().strip(',')) for line in f2]
            #print question[0]
            question_frame = DataFrame(question)
            question_frame = question_frame.drop_duplicates('question_id')
            #print question_frame  # 898 x 2

            user = [json.loads(line.strip().strip(',')) for line in f3]
            user_frame = DataFrame(user)
            user_frame = user_frame.drop_duplicates('user_id')
            #print user_frame    # 3378 x 14

            qtag = [json.loads(line.strip().strip(',')) for line in f4]
            qtag_frame = DataFrame(qtag)
            qtag_frame = qtag_frame.drop_duplicates('question_id')
            #print qtag_frame      # 9999 x 2

            # Generate the source data
            m1 = pd.merge(answer_frame, user_frame, how = 'outer')
            m1 = m1.rename(columns = {'related_id':'question_id'})
            #print m1.head()    # 14781 x 18

            m2 = pd.merge(qtag_frame, question_frame)
            #print m2.head()     # 898 x 3 

            # Generate 
            m = pd.merge(m1, m2, on = 'question_id')
            #print m
            m = m.sort(['question_id','answer_voted'], ascending=[1,0])
            m = m.drop('user_address', axis=1)
            m = m.drop('user_name', axis=1)
            m = m.reset_index(drop=True)
            #print m

            # modify the string in dataframe m
            # m['answer_content'] = str_strip(m['answer_content'])
            # m['question_content'] = str_strip(m['question_content'])
            # m['user_edu'] = str_strip(m['user_edu'])
            # m['user_specialty'] = str_strip(m['user_specialty'])
            # m['user_interest'] = str_strip(m['user_interest'].astype(str))
            # m['user_intro'] = str_strip(m['user_intro'])
            # m['question_tags'] = str_strip(m['question_tags'].astype(str))
            # m['user_recommends'] = str_strip(m['user_recommends'].astype(str))
            
            # print m      # 3195 x 20
            # m.to_json(PATH_OUT + 'total01.json')

            m.to_csv(PATH_OUT + FILE + '_Train.csv', encoding='utf-8', index = False)

In [6]:
def format_test_data(PATH_IN, PATH_OUT, FILE):
    
    with open(PATH_IN + 'answer.json', 'r') as f1, \
        open(PATH_IN + 'question_test.json', 'r') as f2, \
        open(PATH_IN + 'user.json', 'r') as f3, \
        open(PATH_IN + 'qtags.json', 'r') as f4 :
    
            answer = [json.loads(line.strip().strip(',')) for line in f1]
            #print answer[0]
            #answer_frame 
            answer_frame = DataFrame(answer)
            answer_frame = answer_frame.drop_duplicates('answer_id')
            #print answer_frame #14464 x 5

            question = [json.loads(line.strip().strip(',')) for line in f2]
            #print question[0]
            question_frame = DataFrame(question)
            question_frame = question_frame.drop_duplicates('question_id')
            #print question_frame  # 898 x 2

            user = [json.loads(line.strip().strip(',')) for line in f3]
            user_frame = DataFrame(user)
            user_frame = user_frame.drop_duplicates('user_id')
            #print user_frame    # 3378 x 14

            qtag = [json.loads(line.strip().strip(',')) for line in f4]
            qtag_frame = DataFrame(qtag)
            qtag_frame = qtag_frame.drop_duplicates('question_id')
            #print qtag_frame      # 9999 x 2

            # Generate the source data
            m1 = pd.merge(answer_frame, user_frame, how = 'outer')
            m1 = m1.rename(columns = {'related_id':'question_id'})
            #print m1.head()    # 14781 x 18

            m2 = pd.merge(qtag_frame, question_frame)
            #print m2.head()     # 898 x 3 

            # Generate 
            m = pd.merge(m1, m2, on = 'question_id')
            #print m
            m = m.sort(['question_id','answer_voted'], ascending=[1,0])
            m = m.drop('user_address', axis=1)
            m = m.drop('user_name', axis=1)
            m = m.reset_index(drop=True)
            #print m

            # modify the string in dataframe m
            # m['answer_content'] = str_strip(m['answer_content'])
            # m['question_content'] = str_strip(m['question_content'])
            # m['user_edu'] = str_strip(m['user_edu'])
            # m['user_specialty'] = str_strip(m['user_specialty'])
            # m['user_interest'] = str_strip(m['user_interest'].astype(str))
            # m['user_intro'] = str_strip(m['user_intro'])
            # m['question_tags'] = str_strip(m['question_tags'].astype(str))
            # m['user_recommends'] = str_strip(m['user_recommends'].astype(str))
            
            # print m      # 3195 x 20
            # m.to_json(PATH_OUT + 'total01.json')

            m.to_csv(PATH_OUT + FILE + '_Test.csv', encoding='utf-8', index = False)

In [7]:
def Data_Preparing(PATH_IN, PATH_OUT, FILE) :
    
    # RUN the function in pipline
    generate_qapair(PATH_IN, PATH_OUT)
    extract_qid(PATH_IN, PATH_OUT)
    split_qustion(PATH_IN, PATH_OUT)
    format_train_data(PATH_IN, PATH_OUT, FILE)
    format_test_data(PATH_IN, PATH_OUT, FILE)

In [8]:
# FILE = 'Allergy'
# FILE = 'Depression'
# FILE = 'Alzheimer'
# FILE = 'Arthritis'
# PATH_IN  = './' + FILE + '/'
# PATH_OUT = './' + FILE + '/'

# Data_Preparing(PATH_IN, PATH_OUT, FILE)

FILE_LIST = [
    'Allergy',
    'Depression',
    'Alzheimer',
    'Arthritis'
]

for FILE in FILE_LIST :
    PATH_IN  = './' + FILE + '/'
    PATH_OUT = './' + FILE + '/'
    # print PATH_IN
    Data_Preparing(PATH_IN, PATH_OUT, FILE)

/home/zpgao/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:40: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/zpgao/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:39: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [10]:
# File_Type = 'Train'
File_Type = 'Test'


file_list = [   './Allergy/Allergy_' + File_Type +'.csv',
                './Depression/Depression_' + File_Type + '.csv',
                './Arthritis/Arthritis_' + File_Type +'.csv',
                './Alzheimer/Alzheimer_' + File_Type +'.csv' ]

# print train_files

def load_data(file_list):
    with open('./Total_' + File_Type + '.csv','w') as outfile:
        for fname in file_list:
            with open(fname) as infile:
                outfile.write(infile.read())   

load_data(file_list)                

# DO THE TEXT PROCESSING

In [11]:
# Drop duplicates Q-A pair again

Total_Train = pd.read_csv('./Total_Train.csv')     # 7804 rows × 18 columns
Total_Test = pd.read_csv('./Total_Test.csv')       # 50293 rows × 18 columns

Total_Train = Total_Train.drop_duplicates('answer_id')    # 6904 rows × 18 columns
Total_Test = Total_Test.drop_duplicates('answer_id')      # 42513 rows × 18 columns

Total_Train.to_csv('./Total_Train.csv', encoding='utf-8', index = False)
Total_Test.to_csv('./Total_Test.csv', encoding='utf-8', index = False)

In [12]:
# Total_Train.head()
# Total_Train = Total_Train.str.replace(r'[^\x00-\x7f]',' ')

Total_Train = pd.read_csv('./Total_Train.csv')     # 7804 rows × 18 columns
Total_Test = pd.read_csv('./Total_Test.csv')       # 50293 rows × 18 columns

/home/zpgao/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1,2,3,4,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
def Str_Strip(data) :
    data = data.str.replace(r'[^\x00-\x7f]',' ')
    data = data.str.replace('\\n',' ')
    data = data.str.replace('\\r',' ')
    data = data.str.replace(',',' ')
    data = data.str.replace('.',' ')
    data = data.str.replace('"','')
    data = data.str.replace('u\'','\'')
    data = data.str.replace('?','')
    data = data.str.replace('"','')
    data = data.str.replace('.','')
    data = data.str.replace(',','')
    data = data.str.replace('/',' ')
    data = data.str.replace('(','')
    data = data.str.replace(')','')
    data = data.str.replace('[','')
    data = data.str.replace(']','')
    data = data.str.replace('&','')
    data = data.str.replace('\'','')
    data = data.str.replace('!','')
    data = data.str.replace('-','')
    data = data.str.replace(';','')
    data = data.str.lower()   
    return data  

In [14]:
import nltk
import sys
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

def Stemming( Text_DataSet ) :
    for i in range( len(Text_DataSet) ):
        stemmed = []
        # print Text_DataSet[i]
        word_list = Text_DataSet[i].split(' ')
        filtered = [word for word in word_list if word not in stopwords.words('english')]
        # print filtered
        for word in filtered :
            #stemmed.append(PorterStemmer().stem(word))
            stemmed.append(SnowballStemmer("english").stem(word))
        #print stemmed
        stemmed =  [x.encode('UTF8') for x in stemmed]
        # print stemmed
        after_stemmed = ' '.join(stemmed)
        Text_DataSet[i] = after_stemmed

    return Text_DataSet

In [15]:
def Text_Processing(DataSet, ColName) :
    
    Text_DataSet = Str_Strip( DataSet[ColName].astype(str) )
    Text_DataSet = Stemming( Text_DataSet )
    DataSet[ColName] = Text_DataSet 
    
    return DataSet

In [16]:
Text_Col_List = [
    'answer_content',
    'question_content',
    'question_tags',
    
    'user_edu',
    'user_specialty',
    'user_interest',
    'user_intro',
    'user_recommends'
]

for ColName in Text_Col_List :
    
    Total_Train = Text_Processing(Total_Train, ColName)

for ColName in Text_Col_List :
    
    Total_Test = Text_Processing(Total_Test, ColName)

#Total_Train = Text_Processing(Total_Train)
#Total_Test = Text_Processing(Total_Test)

#Total_Train.to_csv('./Total_Train.csv', encoding='utf-8', index = False)
#Total_Test.to_csv('./Total_Test.csv', encoding='utf-8', index = False)

In [17]:
Total_Train.to_csv('./Total_Train_AfterProcessing.csv', encoding='utf-8', index = False)
Total_Test.to_csv('./Total_Test_AfterProcessing.csv', encoding='utf-8', index = False)